In [ ]:
pip install tensorflow
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import json
from google.colab import files
import io

SyntaxError: invalid syntax (1100616390.py, line 1)

In [ ]:
# 1. Memicu jendela unggah file
print("Silakan unggah file JSON Anda (misal: data_chatbot.json)...")
uploaded = files.upload()

# 2. Mendapatkan nama file yang diunggah
# Asumsi Anda hanya mengunggah SATU file JSON
file_name = next(iter(uploaded))
print(f"File '{file_name}' berhasil diunggah.")

# 3. Memuat konten file
# Konten file disimpan dalam bentuk bytes. Kita perlu membacanya.
squad_data = json.load(io.BytesIO(uploaded[file_name]))

# 4. Melanjutkan dengan ekstraksi data seperti sebelumnya
input_texts = []  # Pertanyaan
target_texts = [] # Jawaban

for article in squad_data['data']:
    for paragraph in article['paragraphs']:
        for qa in paragraph['qas']:
            question = qa['question']
            if qa['answers']:
                answer = qa['answers'][0]['text']
                # Tambahkan token <start> dan <end>
                formatted_answer = '<start> ' + answer + ' <end>'

                input_texts.append(question)
                target_texts.append(formatted_answer)

print(f"Total pasangan data (Q-A) yang diekstrak: {len(input_texts)}")
print(f"Contoh Pertanyaan pertama: {input_texts[0]}")

In [ ]:
# --- Konfigurasi (Hemat RAM) ---
MAX_WORDS = 7000     # Batasan Vocabulary
MAX_LEN = 30         # Batasan Panjang Urutan
EMBEDDING_DIM = 50
RNN_UNITS = 128

# 1. Inisialisasi Tokenizer (Asumsi input_texts dan target_texts sudah terisi)
tokenizer = Tokenizer(num_words=MAX_WORDS, filters='')
tokenizer.fit_on_texts(input_texts + target_texts)

# 2. Konversi Teks ke Urutan Angka
encoder_sequences = tokenizer.texts_to_sequences(input_texts)
decoder_sequences = tokenizer.texts_to_sequences(target_texts)

# 3. Padding (Menyamakan panjang urutan)
encoder_input_data = pad_sequences(encoder_sequences, maxlen=MAX_LEN, padding='post')
decoder_input_data = pad_sequences(decoder_sequences, maxlen=MAX_LEN, padding='post')

# 4. MEMBUAT TARGET DECODER: Menggunakan Sparse Integer Targets (Solusi RAM)
# Kita HANYA menyimpan indeks kata, bukan matriks One-Hot yang besar.
# Ini adalah pengganti langsung untuk seluruh Tahap 4 Anda sebelumnya.

# Buat array target integer dengan shape yang sama dengan input decoder
decoder_target_data_sparse = np.zeros_like(decoder_input_data, dtype='int32')

# Looping untuk Pergeseran Target (Shift)
# Target di waktu 't' adalah kata yang ada di decoder_input_data di waktu 't+1'
for i in range(decoder_input_data.shape[0]):
    # Salin semua elemen dari indeks 1 hingga akhir (digeser 1 langkah)
    # Target harus selalu lebih pendek 1 dari input
    decoder_target_data_sparse[i, :-1] = decoder_input_data[i, 1:]

# Ukuran Vocabulary (diperlukan untuk Dense layer)
vocabulary_size = len(tokenizer.word_index) + 1


print("\n--- Hasil Preprocessing ---")
print(f"Ukuran Vocabulary: {vocabulary_size}")
print(f"Shape Encoder Input (Pertanyaan): {encoder_input_data.shape}")
print(f"Shape Decoder Input (Jawaban Input): {decoder_input_data.shape}")
# Perhatikan shape target sekarang: tidak ada lagi dimensi vocabulary_size yang besar!
print(f"Shape Decoder Target (Sparse Integer): {decoder_target_data_sparse.shape}")

In [ ]:
# --- 3.1. ENCODER ---
encoder_inputs = Input(shape=(MAX_LEN,))
encoder_embedding = Embedding(vocabulary_size, EMBEDDING_DIM)(encoder_inputs)

# SimpleRNN Encoder: return_state=True untuk mendapatkan state terakhir (context vector)
encoder_rnn = SimpleRNN(RNN_UNITS, return_state=True)
encoder_outputs, state_h = encoder_rnn(encoder_embedding)
encoder_states = state_h # Context vector dari input

# --- 3.2. DECODER ---
decoder_inputs = Input(shape=(MAX_LEN,))
decoder_embedding = Embedding(vocabulary_size, EMBEDDING_DIM)(decoder_inputs)

# SimpleRNN Decoder: return_sequences=True untuk output prediksi di setiap langkah waktu
decoder_rnn = SimpleRNN(RNN_UNITS, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_rnn(decoder_embedding, initial_state=encoder_states)

# Dense Layer untuk memprediksi kata (softmax di atas vocabulary)
decoder_dense = Dense(vocabulary_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# --- 3.3. TRAINING MODEL LENGKAP ---
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Kompilasi Model dengan Sparse Categorical Crossentropy
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print("\n--- Ringkasan Model Pelatihan ---")
model.summary()

# --- 3.4. INFERENSI MODEL ---
# Model Encoder Inferensi (Input: Pertanyaan -> Output: Context Vector)
decoder_inputs_single = Input(shape=(1,), name='decoder_input_single')

# Input State untuk Decoder Inferensi
decoder_state_input = Input(shape=(RNN_UNITS,))

# Decoder Inferensi (Input: 1 Token & Previous State -> Output: Prediksi 1 Token & New State)
decoder_outputs, state_h_output = decoder_rnn(
    decoder_embedding, initial_state=decoder_state_input
)
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs, decoder_state_input],
    [decoder_outputs, state_h_output]
)

In [ ]:
# Training
EPOCHS = 10
BATCH_SIZE = 64

print("\nMemulai Pelatihan Model Simple RNN...")
history = model.fit(
    # Input Encoder (Pertanyaan) dan Input Decoder (Jawaban yang digeser)
    [encoder_input_data, decoder_input_data],

    # Target (Jawaban yang digeser 1 langkah, dalam bentuk Sparse Integer)
    decoder_target_data_sparse,

    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.2 # Menggunakan 20% data untuk validasi
)
print("Pelatihan Selesai!")

In [ ]:

# Asumsi: tokenizer, MAX_LEN, encoder_inputs, encoder_states, RNN_UNITS, decoder_rnn, decoder_dense sudah didefinisikan dari blok sebelumnya.

# --- 1. DEFINISI MODEL INFERENSI (Wajib di sini untuk menghindari NameError) ---
# KODE DI BAWAH INI MEMBUTUHKAN VARIABEL DARI BLOK MODELLING (rnn_seq2seq_model.py)

# Model Encoder Inferensi (Input: Pertanyaan -> Output: Context Vector)
# KODE INI MENGGUNAKAN LAYER DAN INPUT DARI BLOK MODELLING
encoder_model = Model(encoder_inputs, encoder_states)

# Input State untuk Decoder Inferensi
decoder_state_input = Input(shape=(RNN_UNITS,), name='decoder_state_input')
decoder_inputs_single = Input(shape=(1,), name='decoder_input_single')
decoder_embedding_inference = encoder_model.get_layer(name='embedding')(decoder_inputs_single) # Asumsi layer embedding dinamai 'embedding_1'

# Decoder Inferensi (Input: 1 Token & Previous State -> Output: Prediksi 1 Token & New State)
decoder_outputs, state_h_output = decoder_rnn(
    decoder_embedding_inference, initial_state=decoder_state_input
)
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs_single, decoder_state_input],
    [decoder_outputs, state_h_output]
)
# --- AKHIR DEFINISI MODEL INFERENSI ---

# Memetakan indeks token ke kata (untuk output)
reverse_word_index = dict((i, word) for word, i in tokenizer.word_index.items())

# --- 2. FUNGSI DECODING ---
def decode_sequence(input_seq):
    """
    Fungsi untuk menghasilkan urutan jawaban dari urutan pertanyaan.
    Menggunakan Model Inferensi (Encoder-Decoder) secara langkah demi langkah.
    """

    # 1. Encode input dan dapatkan initial state (context vector)
    states_value = encoder_model.predict(input_seq)

    # 2. Inisialisasi input target dengan token <start>
    target_seq = np.zeros((1, 1))

    start_token_index = tokenizer.word_index.get('start')
    if start_token_index is None:
        print("Error: Token '<start>' tidak ditemukan di vocabulary.")
        return ""

    target_seq[0, 0] = start_token_index

    # 3. Looping Decoding Kata demi Kata
    stop_condition = False
    decoded_sentence = ''

    max_len_limit = MAX_LEN

    while not stop_condition:
        # Prediksi output (token berikutnya) dan state baru
        output_tokens, h = decoder_model.predict([target_seq, states_value])

        # Ambil token dengan probabilitas tertinggi (argmax)
        sampled_token_index = np.argmax(output_tokens[0, 0, :])
        sampled_word = reverse_word_index.get(sampled_token_index, '')

        # Cek kondisi berhenti
        if (sampled_word == 'end' or len(decoded_sentence.split()) >= max_len_limit):
            stop_condition = True
        elif sampled_word not in ['start', '']:
            decoded_sentence += ' ' + sampled_word

        # Perbarui input target (kata yang baru diprediksi menjadi input decoder berikutnya)
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Perbarui state (h) untuk langkah waktu berikutnya
        states_value = h

    return decoded_sentence.strip()

In [ ]:
# Perlu menginstal NLTK di Colab:
!pip install nltk # Perintah paksa untuk memastikan semua dependensi terinstal
import numpy as np
import random
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.model_selection import train_test_split
# Import pad_sequences dari lokasi yang lebih stabil
import tensorflow as tf
from tensorflow.keras.utils import pad_sequences

# --- PENTING: MENAMBAHKAN PATH DATA NLTK SECARA EKSPLISIT ---
# Ini adalah solusi untuk mengatasi LookupError: Resource punkt_tab not found di Colab
import nltk.data
nltk.data.path.append('/root/nltk_data')

# --- PENTING: PASTIKAN DOWNLOAD NLTK BERHASIL (Mengatasi punkt_tab error) ---
try:
    # Mengunduh secara eksplisit di awal dan menambahkan paket yang sering dibutuhkan
    nltk.download('punkt', quiet=False)
    nltk.download('wordnet', quiet=False)
    # Download paket lain yang mungkin menggunakan punkt_tab secara internal
    nltk.download('averaged_perceptron_tagger', quiet=False)
except Exception as e:
    print(f"Peringatan: Gagal mendownload NLTK. Evaluasi mungkin gagal. Error: {e}")

# --- ASUMSI VARIABEL GLOBAL SUDAH TERSEDIA ---
# model, tokenizer, MAX_LEN, input_texts, target_texts, dll.

# Asumsi fungsi decode_sequence dan variabel global lainnya sudah ada dari full_inference.py

# Membagi data menjadi training dan testing (90% training, 10% testing)
# Ini dilakukan agar evaluasi menggunakan data yang belum pernah dilihat model.
try:
    if 'input_texts' not in globals() or not input_texts:
        raise NameError("Variabel input_texts belum dimuat.")

    # Memastikan data testing menggunakan input_texts, bukan input_texts_train
    # Membagi 10% data untuk testing
    input_train, input_test, target_train, target_test = train_test_split(
        input_texts, target_texts, test_size=0.1, random_state=42
    )

    # Pre-process data testing
    encoder_test_sequences = tokenizer.texts_to_sequences(input_test)
    encoder_test_data = pad_sequences(encoder_test_sequences, maxlen=MAX_LEN, padding='post')

    print(f"\nTotal sampel untuk Evaluasi (Testing): {len(encoder_test_data)}")

except NameError as e:
    print(f"Error: {e}. Pastikan Anda menjalankan Preprocessing (Langkah 1) terlebih dahulu.")
    exit() # Menghentikan eksekusi jika data belum siap


# --- FUNGSI BANTU UNTUK BLEU SCORE ---
# SmoothingFunction untuk menangani kasus 0-matching (agar tidak error ZeroDivisionError)
smoothie = SmoothingFunction().method1

def calculate_bleu(reference, candidate):
    """Menghitung BLEU Score untuk satu pasangan jawaban."""

    # Tokenisasi Jawaban Asli (Referensi) dan Jawaban Model (Kandidat)
    # Jawaban Asli harus berupa list of lists of tokens
    reference_tokens = [nltk.word_tokenize(ref) for ref in reference.split(' | ')]
    candidate_tokens = nltk.word_tokenize(candidate)

    # Hitung BLEU Score
    # Sentence BLEU digunakan untuk membandingkan satu kalimat
    try:
        score = sentence_bleu(reference_tokens, candidate_tokens, smoothing_function=smoothie)
    except Exception:
        # Menghandle error jika tokenization gagal pada sampel tertentu
        return 0.0
    return score

# --- RUN EVALUASI DAN HITUNG BLEU SCORE RATA-RATA ---
bleu_scores = []
num_samples = len(encoder_test_data)
# Gunakan hanya 100 sampel pertama agar tidak terlalu lama (tergantung ukuran data Anda)
max_samples_to_test = min(num_samples, 5)

print(f"\nMemulai evaluasi pada {max_samples_to_test} sampel...")

for i in range(max_samples_to_test):
    # Ambil input (pertanyaan) dari data testing
    input_seq = np.expand_dims(encoder_test_data[i], axis=0)

    # Dapatkan Jawaban Model (prediksi)
    predicted_answer_raw = decode_sequence(input_seq)

    # Dapatkan Jawaban Asli (referensi)
    # Hapus token <start> dan <end> dari target_test
    reference_answer_raw = target_test[i].replace('<start>', '').replace('<end>', '').strip()

    # Bersihkan Jawaban Model (hapus padding, karakter berulang, dll.)
    predicted_answer = predicted_answer_raw.replace('<end>', '').strip()

    # Tokenisasi dan Hitung BLEU
    bleu_score_sample = calculate_bleu(reference_answer_raw, predicted_answer)
    bleu_scores.append(bleu_score_sample)

    # Tampilkan contoh setiap 20 sampel
    if (i + 1) % 20 == 0 or i == max_samples_to_test - 1:
        print(f"\n--- Contoh Sampel {i+1} ---")
        print(f"Pertanyaan: {input_test[i]}")
        print(f"Jawaban Model: {predicted_answer}")
        print(f"Jawaban Asli: {reference_answer_raw}")
        print(f"BLEU Score Sampel: {bleu_score_sample:.4f}")

# --- HASIL AKHIR ---
final_bleu_score = np.mean(bleu_scores)

print("\n================================================")
print(f"FINAL BLEU SCORE RATA-RATA ({max_samples_to_test} Sampel): {final_bleu_score:.4f}")
print("================================================")


In [ ]:
import matplotlib.pyplot as plt

# Mengekstrak data dari objek history
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
accuracy = history_dict['accuracy']
val_accuracy = history_dict['val_accuracy']

epochs_range = range(1, len(accuracy) + 1)

# Membuat plot untuk Loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, loss_values, 'bo', label='Training Loss')
plt.plot(epochs_range, val_loss_values, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Membuat plot untuk Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs_range, accuracy, 'ro', label='Training Accuracy')
plt.plot(epochs_range, val_accuracy, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()